# Scraping Top Repositories for Topics on GitHub

Introduction
- Web scraping is a technique for extracting information from websites
- GitHub is a web-based interface that uses Git, the open source version control software that lets multiple people make seperate changes to web pages at the same time.
- We use different tools in this code like Python, requests, Beautiful soup, Pandas for scraping

Here are the steps we'll follow:

- We're going to scrape https://github.com/topics
- We'll get a list of topics. For each topic, we'll get topic title, topic page URL and topic description
- For each topic, we'll get the top 25 repositories in the topic from the topic page
- For each repository, we'll grab the repo name, username, stars and repo naurl
- For each topic we'll create a CSV file in the following format:

```
Repo Name,User Name,Stars,Repo URL
three.jb,mrdoob,69700,https://github.com/mrdoob/three.js
libgdx,libgdx,18300,https://github.com/libgdx/libgdx
```

## Scrape the list of topics from GitHub

- Use requests to download the page
- User BS4 to parse and extract information
- Convert to a Pandas dataframe



To download the page:

In [27]:
import requests
from bs4 import BeautifulSoup

def get_topics_page():
    # TODO - add comments
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'. format(topic_url))  
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [2]:
doc = get_topics_page()

In [3]:
doc.find('a')

<a class="px-2 py-4 color-bg-info-inverse color-text-white show-on-focus js-skip-to-content" href="#start-of-content">Skip to content</a>

Let's create some helper functions to parse information from the page.

To get topic titles, we can pick `p` tags with the `class` ...

![](https://i.imgur.com/xuT6n6J.png)

In [28]:
def get_topic_titles(doc):
        selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
        topic_title_tags = doc.find_all('p',{'class': selection_class})
        topic_titles = []
        for tag in topic_title_tags:
            topic_titles.append(tag.text)
        return topic_titles
        pass
    


get_topic_titles can be used to get the list of titles

In [5]:
titles = get_topic_titles(doc)

In [6]:
len(titles)

30

In [7]:
titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

Similarly we have defined functions for descriptions and URLs

In [29]:
def get_topic_descs(doc):
    desc_selector = 'f5 color-text-secondary mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': desc_selector})
    topic_descs = []
    for tag in topic_desc_tags:
            topic_descs.append(tag.text.strip())
    return topic_descs

In [30]:
def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a', {'class': 'd-flex no-underline'})
    topic_urls = []
    base_url = 'https://github.com'
    for tag in topic_link_tags:
            topic_urls.append(base_url + tag['href'])

    return topic_urls


Let's put this all together into a single function

In [31]:
def scrape_topics():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'. format(topic_url))   
    topics_dict = {
        'title': get_topic_titles(doc),
        'description': get_topic_descs(doc),
        'url': get_topic_urls(doc)
    }
    return pd.DataFrame(topics_dict)

## Get the top 25 repositories from the topic page

- Download the page
- check successful  response
- parse using Beautiful Soup

In [32]:
def get_topic_page(topic_url):
     #Download the page
    response = requests.get(topic_url)
    # Check successful response
    if response.status_code !=200:
        raise Exception('Failed to load page {}'. format(topic_url))
    # parse using Beautiful soup    
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc


In [33]:
doc = get_topic_page('https://github.com/topics/3d')

The h1 tag, will help in getting the name of sub topic of particular topic

In [34]:
def get_repo_info(h1_tag,star_tag):
    # returns all the required info about a repository
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tags[0].text.strip())
    return username, repo_name, stars, repo_url

In [35]:
def get_topic_repos(topic_doc):
     # Get the h1 tags containing repo title, repo URL and username
    h1_selection_class = 'f3 color-text-secondary text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h1', {'class': h1_selection_class} )
    #Get star tags
    star_tags = topic_doc.find_all('a', {'class': 'social-count float-none'})
    
    topic_repos_dict = {'username': [],'repo_name':[],'stars':[],'repo_url': []}
    #Get repo info
    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i],star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])
        
    return pd.DataFrame(topic_repos_dict)


In [36]:
def scrape_topic(topic_url, path):
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(path, index=None)

## Putting it all together

- We have a function to get the list of topics
- We have a function to create a CSV file for scrape repos from a topics page
- Let's create a function to put them together

In [37]:
def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics()
    
    os.makedirs('data', exist_ok=True)
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'. format(row['title']))
        scrape_topic(row['url'], 'data/{}.csv'. format(row['title']))

Let's run it to scrape the top repos for the all the topics on the first page of https://github.com/topics

We can check that the CSVs were created properly

In [38]:
scrape_topics_repos

<function __main__.scrape_topics_repos()>

In [39]:
# read and display CSV using Pandas

## References

- https://www.pluralsight.com/guides/web-scraping-with-beautiful-soup
- https://www.jcchouinard.com/web-scraping-with-python-and-requests-html/
- https://github.com/topics
- https://imgur.com/

In [7]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
